In [1]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import gzip
import Preprocessing as pproc 

directory = os.getcwd()
data_path = os.path.join(directory, 'Data')

# Definining the path for the files
RB_path = os.path.join(data_path, 'RateBeer')
BA_path = os.path.join(data_path, 'BeerAdvocate')
MB_path = os.path.join(data_path, 'matched_beer_data')

In [2]:
#Transformation of the .txt files
pproc.text_to_csv(RB_path, 'ratings')
pproc.text_to_csv(RB_path, 'reviews')

pproc.text_to_csv(BA_path, 'ratings')
pproc.text_to_csv(BA_path, 'reviews')

pproc.text_to_csv(MB_path, 'ratings_ba')
pproc.text_to_csv(MB_path, 'ratings_with_text_ba')
pproc.text_to_csv(MB_path, 'ratings_rb')
pproc.text_to_csv(MB_path, 'ratings_with_text_rb')

In [3]:
#MATCHING BEERS:
MB_beers = pd.read_csv(MB_path + '/beers.csv', header=1)
MB_breweries = pd.read_csv(MB_path + '/breweries.csv', header=1)
MB_users = pd.read_csv(MB_path + '/users.csv', header=1)
MB_users_approx = pd.read_csv(MB_path + '/users_approx.csv', header=1)
MB_ratings = pd.read_csv(MB_path + '/ratings.csv', header=1)


In [4]:
#RATEBEERS WEB:
RB_beers = pd.read_csv(RB_path + '/beers.csv')
RB_breweries = pd.read_csv(RB_path + '/breweries.csv')
RB_users = pd.read_csv(RB_path + '/users.csv')

In [15]:
#BEERADVOCATES WEB:
BA_beers = pd.read_csv(BA_path + '/beers.csv')
BA_breweries = pd.read_csv(BA_path + '/breweries.csv')
BA_users = pd.read_csv(BA_path + '/users.csv')
BA_ratings = pd.read_csv(BA_path + '/ratings.tsv', sep='\t')
BA_reviews = pd.read_csv(BA_path + '/reviews.tsv', sep='\t')

In [5]:
MB_ratings

,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,...,date.1,overall.1,palate.1,rating.1,style.1,taste.1,text.1,user_id.1,user_name.1,year
0,11.3,4.50,4.50,645,Trappistes Rochefort 10,207,Brasserie de Rochefort,2011-12-25 11:00:00,5.0,4.50,...,1387710000,19.0,4.0,4.6,Abt/Quadrupel,9.0,a) Geruch malzig-schwer-sÃ¼Ã. Riecht schon ...,83106,Erzengel,2011
1,5.0,NaN,NaN,28191,Myanmar Lager Beer,9369,Myanmar Brewery and Distillery,2011-11-30 11:00:00,NaN,NaN,...,1322564400,6.0,2.0,1.7,Pale Lager,4.0,"Can. Weak and watery, not the best beer of the...",91324,visionthing,2011
2,5.0,3.50,3.50,57911,Cantillon Tyrnilambic Baie D’Argousier Lambic,388,Brasserie Cantillon,2012-08-04 10:00:00,4.0,4.00,...,1353582000,17.0,4.0,4.1,Lambic Style - Fruit,8.0,"Bottle @ One Pint Pub, Helsinki. Originally ra...",98624,tiong,2012
3,5.0,4.00,3.50,57913,Cantillon Pikkulinnun Viskilambic,388,Brasserie Cantillon,2012-08-04 10:00:00,4.0,4.00,...,1416222000,16.0,4.0,4.1,Lambic Style - Unblended,9.0,"Draught @Â Pikkulintu, Helsinki, Finland. A pr...",98624,tiong,2012
4,6.0,4.00,4.00,81125,Drie Fonteinen Oude Geuze - Armand & Tommy,2216,Brouwerij 3 Fonteinen,2012-08-29 10:00:00,4.0,4.00,...,1345284000,16.0,4.0,4.0,Lambic Style - Gueuze,8.0,750ml bottleBottling date: 2011/02/17 - Pours ...,98624,tiong,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21959,8.5,4.50,3.50,28030,Valeir Divers,3463,Brouwerij Contreras,2011-12-15 11:00:00,4.0,4.00,...,1323946800,15.0,4.0,3.7,Abbey Tripel,7.0,Valeir Divers 33cl bottle from www.belgianbeer...,137922,TheBeerWatcher,2011
21960,8.0,4.25,4.25,5057,Fantôme Saison,738,Brasserie Fantôme,2017-01-10 11:00:00,4.0,4.00,...,1487502000,16.0,4.0,4.0,Saison,8.0,.................................................,394232,jonj,2017
21961,5.2,NaN,NaN,349,Jupiler,134,Brasserie Piedboeuf,2014-02-24 11:00:00,NaN,NaN,...,1393153200,1.0,1.0,1.2,Pale Lager,2.0,"Biere, foin, pain grillÃ©, terreux, banane, ca...",304802,maxilouis,2014
21962,6.5,3.00,3.00,127162,Redenaar,32560,Brouwerij d'Oude Maalderij,2014-12-28 11:00:00,3.0,2.75,...,1397037600,17.0,3.0,3.9,Belgian Ale,8.0,This is the first blond ale from this brewery....,276666,simoen,2014


In [6]:
MB_ratings['date'] =pd.to_datetime(MB_ratings.date, unit='s')
MB_ratings['year'] = MB_ratings['date'].dt.year
columns_to_keep = ['user_id', 'rating','year', 'beer_name']
filter_MB_ratings = MB_ratings.filter(columns_to_keep)


In [7]:
filter_MB_ratings.head(2)

,user_id,rating,year,beer_name
0,erzengel.248045,4.8,2011,Trappistes Rochefort 10
1,visionthing.639993,3.0,2011,Myanmar Lager Beer


So with the previous datasets we have the geographical location and the brewery name. Furthermore, it can be merged by the userID that coincides in both columns.

In [8]:
geo_beers = pd.merge(filter_MB_ratings, MB_users[['user_id','location']], on='user_id', how='outer')

In [9]:
geo_beers

,user_id,rating,year,beer_name,location
0,erzengel.248045,4.80,2011.0,Trappistes Rochefort 10,Germany
1,visionthing.639993,3.00,2011.0,Myanmar Lager Beer,Estonia
2,tiong.608427,3.85,2012.0,Cantillon Tyrnilambic Baie D’Argousier Lambic,Finland
3,tiong.608427,3.68,2012.0,Cantillon Pikkulinnun Viskilambic,Finland
4,tiong.608427,4.00,2012.0,Drie Fonteinen Oude Geuze - Armand & Tommy,Finland
...,...,...,...,...,...
24100,shadesofgray.371155,NaN,NaN,NaN,"United States, Pennsylvania"
24101,ypsifly.8833,NaN,NaN,NaN,"United States, Michigan"
24102,hardtarget.1712,NaN,NaN,NaN,"United States, Texas"
24103,nelsonsaigon.897449,NaN,NaN,NaN,Vietnam
